In [94]:
import pandas as pd
import numpy as np

from spacy.lang.en import English

import torch
from allennlp.modules.elmo import Elmo, batch_to_ids

In [220]:
# from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.Defaults.create_tokenizer(nlp)

def tokenize_str(text: str, tokenizer):
    tokenized_str = map(str, list(tokenizer(text)))
    return tokenized_str

# options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
# weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
# elmo = Elmo(options_file, weight_file, 1, dropout=0)

def elmo_embed_words(tokenized, elmo):
    character_ids = batch_to_ids(tokenized)
    embeddings = elmo(character_ids)
    
    return embeddings['elmo_representations'][0]

def average_embedding_from_embedding_list(embedding):
    numnonzero = (embedding.sum(axis=-1) != 0).sum()
    summedembedding = embedding.sum(axis=0).sum(axis=0)
    return summedembedding / numnonzero

def average_embedding_of_text(text: str, tokenizer, elmo):
    tokenized_str = tokenize_str(text, tokenizer)
    elmo_wordembeddings = elmo_embed_words(tokenized_str, elmo)
    return average_embedding_from_embedding_list(elmo_wordembeddings).detach().numpy()

In [21]:
# nyt_list = pd.read_pickle('../../NYTcorpus_updated.p')
# nyt_np = np.array(nyt_list)  # this actually ends up being 1d, the individual items remain lists
# nyt_df = pd.DataFrame(nyt_list)
nyt_df = pd.DataFrame(pd.read_pickle('../../NYTcorpus_updated.p'))

In [221]:
emb_arr = pd.DataFrame()
for text in smallnytdf[2]:
    textembed_df = pd.DataFrame(average_embedding_of_text(text, tokenizer, elmo)).T
    textembed_df.to_csv('nytcorpuselmo.csv', mode='a', header=None)

             0
0     0.019487
1     0.016164
2     0.050672
3    -0.134242
4     0.114453
...        ...
1019 -0.325436
1020  0.481848
1021 -0.181163
1022 -0.294532
1023  0.148607

[1024 rows x 1 columns]
             0
0    -0.109163
1     0.020019
2    -0.070485
3    -0.092134
4     0.275918
...        ...
1019 -0.283050
1020  0.397744
1021 -0.126029
1022 -0.137895
1023  0.078819

[1024 rows x 1 columns]
             0
0    -0.333518
1    -0.119705
2    -0.192843
3    -0.079650
4     0.435958
...        ...
1019 -0.199283
1020  0.154433
1021  0.013768
1022  0.123915
1023 -0.078641

[1024 rows x 1 columns]
             0
0    -0.038084
1    -0.130157
2    -0.134834
3     0.022552
4     0.272745
...        ...
1019 -0.240587
1020  0.412710
1021 -0.205257
1022 -0.342578
1023 -0.059370

[1024 rows x 1 columns]
             0
0     0.032344
1     0.036680
2     0.086842
3    -0.133328
4     0.102461
...        ...
1019 -0.304232
1020  0.433974
1021 -0.188966
1022 -0.268570
1023  0.199765


In [164]:
happy = smallnytdf[2].apply(lambda text: elmo_embed_words(list(tokenize_str(text, tokenizer)), elmo))

In [226]:
pd.DataFrame(np.array([1,2,3,4])).T

,0,1,2,3
0,1,2,3,4


In [193]:
finalsmallnytdf = happy.apply(lambda embeddings: average_embedding_from_embedding_list(embeddings))

In [225]:
np.expand_dims(finalsmallnytdf[0].detach().numpy(), axis=0)

array([[-0.04805946,  0.00796197,  0.09796102, ..., -0.19531074,
        -0.29650235,  0.16922697]], dtype=float32)

In [206]:
np.array(list(finalsmallnytdf.apply(lambda pytensor: pytensor.detach().numpy().tolist())))

(5, 1024)

In [201]:
finalsmallnytdf.apply(lambda pytensor: list(pytensor))

0    [tensor(-0.0481, grad_fn=<SelectBackward>), te...
1    [tensor(-0.1290, grad_fn=<SelectBackward>), te...
2    [tensor(-0.3453, grad_fn=<SelectBackward>), te...
3    [tensor(-0.0384, grad_fn=<SelectBackward>), te...
4    [tensor(0.0323, grad_fn=<SelectBackward>), ten...
Name: 2, dtype: object

In [177]:
(finalsmallnytdf[0])

tensor([-0.0481,  0.0080,  0.0980,  ..., -0.1953, -0.2965,  0.1692],
       grad_fn=<DivBackward0>)

In [178]:
finalsmallnytdf[1]

tensor([-0.1290,  0.0300, -0.0576,  ..., -0.1271, -0.1411,  0.0881],
       grad_fn=<DivBackward0>)

In [179]:
finalsmallnytdf[2]

tensor([-0.3453, -0.1202, -0.1831,  ...,  0.0131,  0.1221, -0.0737],
       grad_fn=<DivBackward0>)

In [180]:
finalsmallnytdf[3]

tensor([-0.0384, -0.1298, -0.1349,  ..., -0.2053, -0.3426, -0.0594],
       grad_fn=<DivBackward0>)

In [165]:
list(map(lambda x: x.shape, happy))

[torch.Size([98, 13, 1024]),
 torch.Size([161, 13, 1024]),
 torch.Size([222, 14, 1024]),
 torch.Size([80, 10, 1024]),
 torch.Size([98, 11, 1024])]

In [174]:
(happy[0].sum(axis=-1) != 0).sum()

tensor(427)

In [167]:
happy[1]

tensor([[[-1.0770e+00, -1.0845e+00, -5.5063e-01,  ...,  2.2304e-02,
          -5.5962e-01, -2.4252e-01],
         [ 3.3265e-02, -7.5589e-01, -4.7273e-01,  ..., -2.6711e-01,
          -6.8731e-01, -4.7712e-01],
         [ 2.7323e-01, -6.5155e-01, -6.5962e-01,  ...,  7.1965e-02,
          -4.0458e-01, -7.4857e-01],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-3.8098e-01,  1.8938e-01, -3.6145e-01,  ...,  9.7992e-02,
          -5.7893e-02,  7.3032e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0

In [168]:
happy[0] == happy[1]

RuntimeError: The size of tensor a (98) must match the size of tensor b (161) at non-singleton dimension 0

In [122]:
options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

# Compute two different representation for each token.
# Each representation is a linear weighted combination for the
# 3 layers in ELMo (i.e., charcnn, the outputs of the two BiLSTM))
elmo = Elmo(options_file, weight_file, 1, dropout=0)

# use batch_to_ids to convert sentences to character ids
sentences = [['First', 'sentence', '.'], ['Another', '.']]
character_ids = batch_to_ids(sentences)

embeddings = elmo(character_ids)
embeddings

# embeddings['elmo_representations'] is length two list of tensors.
# Each element contains one layer of ELMo representations with shape
# (2, 3, 1024).
#   2    - the batch size
#   3    - the sequence length of the batch
#   1024 - the length of each ELMo vector

{'elmo_representations': [tensor([[[ 0.1474, -0.1475,  0.1376,  ...,  0.0270, -0.4051, -0.0498],
           [ 0.2394,  0.0769,  0.4126,  ..., -0.1671, -0.1707,  0.3884],
           [-0.7602, -0.4944, -0.5355,  ..., -0.0803,  0.0361,  0.1128]],
  
          [[ 0.2603, -0.4437,  0.2726,  ..., -0.0830, -0.1522, -0.1361],
           [-0.7772, -0.4294, -0.2651,  ..., -0.0803,  0.0361,  0.1128],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
         grad_fn=<CopySlices>)],
 'mask': tensor([[ True,  True,  True],
         [ True,  True, False]])}

In [123]:
# tenstest = 
embeddings['elmo_representations'][0]

tensor([[[ 0.1474, -0.1475,  0.1376,  ...,  0.0270, -0.4051, -0.0498],
         [ 0.2394,  0.0769,  0.4126,  ..., -0.1671, -0.1707,  0.3884],
         [-0.7602, -0.4944, -0.5355,  ..., -0.0803,  0.0361,  0.1128]],

        [[ 0.2603, -0.4437,  0.2726,  ..., -0.0830, -0.1522, -0.1361],
         [-0.7772, -0.4294, -0.2651,  ..., -0.0803,  0.0361,  0.1128],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<CopySlices>)

In [121]:
tenstest.sum(axis=0).sum(axis=0) 

tensor([-0.8903, -1.4381,  0.0221,  ..., -0.3837, -0.6559,  0.4280],
       grad_fn=<SumBackward1>)

In [103]:
tenstest

tensor([[[ 0.1474, -0.1475,  0.1376,  ...,  0.0270, -0.4051, -0.0498],
         [ 0.2394,  0.0769,  0.4126,  ..., -0.1671, -0.1707,  0.3884],
         [-0.7602, -0.4944, -0.5355,  ..., -0.0803,  0.0361,  0.1128]],

        [[ 0.2603, -0.4437,  0.2726,  ..., -0.0830, -0.1522, -0.1361],
         [-0.7772, -0.4294, -0.2651,  ..., -0.0803,  0.0361,  0.1128],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<CopySlices>)

tensor([[ 0.4077, -0.5912,  0.4101,  ..., -0.0560, -0.5573, -0.1860],
        [-0.5377, -0.3524,  0.1475,  ..., -0.2474, -0.1347,  0.5012],
        [-0.7602, -0.4944, -0.5355,  ..., -0.0803,  0.0361,  0.1128]],
       grad_fn=<AddBackward0>)

In [14]:
smallnytdf = nyt_df.head()

In [71]:
tokenizer = SpacyTokenizer("en_core_web_sm")

error: bad escape \p at position 257

In [86]:
# from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer
# tokenizer = SpacyTokenizer("en_core_web_sm")
# tokenizer.tokenize(nyt_df.iloc[1,2])

# from spacy.lang.en import English
# nlp = English()
# # Create a Tokenizer with the default settings for English
# # including punctuation rules and exceptions
# tokenizer = nlp.Defaults.create_tokenizer(nlp)

# nyt_tokenized = list(tokenizer(nyt_df.iloc[1,2]))

# from nltk.tokenize import WordPunctTokenizer
# WordPunctTokenizer().tokenize(nyt_df.iloc[1,2])

In [45]:
nyt_charids = batch_to_ids(nyt_df.iloc[1,2])

In [76]:
nyt_df.iloc[1,2]

"LEAD: *3*** COMPANY REPORTS ** *3*NORTHERN INDIANA PUBLIC SERVICE CO (NYSE) Qtr to Dec 31 1986 1985 Revenue 399,038,000 522,750,000 Net inc 22,007,000 c61,614,000 Share earns .21 - Yr rev 1,626,127,000 1,908,503,000 Net loss b14,117,000 d12,298,000 *3*** COMPANY REPORTS ** *3*NORTHERN INDIANA PUBLIC SERVICE CO (NYSE) Qtr to Dec 31 1986 1985 Revenue 399,038,000 522,750,000 Net inc 22,007,000 c61,614,000 Share earns .21 - Yr rev 1,626,127,000 1,908,503,000 Net loss b14,117,000 d12,298,000 b-Includes a $102 million after-tax charge related to a settlement of a coal contract case. c-Net loss  d-Net income  The year-ago periods' net loss and net income figures were after an extraordinary charge of $94,843,000 as a writeoff of the company's cancelled Dailly N-1 plant. Preferred dividend requirements exceeded the final net in 1985."

In [79]:
nyt_charids = batch_to_ids(map(str, nyt_tokenized))

tensor([[[259,  77, 260,  ..., 261, 261, 261],
         [259,  70, 260,  ..., 261, 261, 261],
         [259,  66, 260,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[259,  59, 260,  ..., 261, 261, 261],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[259,  43, 260,  ..., 261, 261, 261],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        ...,

        [[259, 106, 260,  ..., 261, 261, 261],
         [259, 111, 260,  ..., 261, 261, 261]